# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [32]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('desaster_response', con=engine)
X = df["message"]
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [34]:
def tokenize(text):
    return word_tokenize(text)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [35]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

In [36]:
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [58]:
from sklearn.metrics import f1_score

y_pred = pipeline.predict(X_test)

# define f1 score for mulit output

# f1_score = f1_score(
#     y_true=y_test, 
#     y_pred=y_pred, 
#     average=None, 
# )

In [57]:
f1_score

array([ 0.48023426,  0.        ,  0.59399898,  0.12977099,  0.1504065 ,
        0.05217391,  0.        ,  0.11764706,  0.        ,  0.3030303 ,
        0.52970651,  0.31304348,  0.07518797,  0.08737864,  0.        ,
        0.06143345,  0.19438445,  0.05699916,  0.00350877,  0.18421053,
        0.16237624,  0.11282051,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.64377904,  0.47295864,  0.46193772,
        0.02247191,  0.73676471,  0.15609756,  0.075     ,  0.43238587])

In [55]:
from sklearn.metrics import classification_report

target_names = Y.columns.tolist()

print(classification_report(y_true=y_test, y_pred=y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               request       0.85      0.33      0.48      1469
                 offer       0.00      0.00      0.00        37
           aid_related       0.74      0.49      0.59      3546
          medical_help       0.64      0.07      0.13       706
      medical_products       0.79      0.08      0.15       445
     search_and_rescue       0.55      0.03      0.05       219
              security       0.00      0.00      0.00       161
              military       0.56      0.07      0.12       274
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.18      0.30       546
                  food       0.83      0.39      0.53       953
               shelter       0.85      0.19      0.31       750
              clothing       0.71      0.04      0.08       126
                 money       0.64      0.05      0.09       192
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
parameters = {
        'clf__min_samples_split': [5,10, 15],
        'clf__n_estimators': [50, 100, 150]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='accuracy', verbose=1, n_jobs=-1)

NameError: name 'GridSearchCV' is not defined

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
file_path = 'tbd'

with open(file_path, 'wb') as file:  
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.